In [3]:
import os
import datetime
import polars as pl
import pandas as pd
pl.Config(tbl_rows=100)
from src.najdi_rok import najdi_rok
from src.pocet_stran import pocet_stran
from src.bez_bordelu import bez_bordelu
from src.zjisti_vazbu import zjisti_vazbu

In [5]:
df = pl.read_parquet(os.path.join("data/cnb_sloupce","100.parquet"))
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","leader.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","008.parquet")), left_on="001", right_on="001", how="left")
df = df.to_pandas()
df = df[df["leader"].str[6].isin(["a", "t"])]
df = df[~df["leader"].str[7].isin(["b", "i", "s", " "])]
df = df[(df["008"].str[15:17] == "xr") & (df["008"].str[35:38] == "cze")]
df = pl.from_pandas(df)
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","020.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","022.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","245.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","300.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","655.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","700.parquet")), left_on="001", right_on="001", how="left")
df = df.explode("022_a").filter(pl.col("022_a").is_null())
df = df.with_columns(pl.col('008').map_elements(najdi_rok, return_dtype=int).alias('rok'))
df = df.with_columns(pl.col('300_a').map_elements(pocet_stran, return_dtype=int).alias('stran'))
df = df.with_columns(pl.col('245_a').map_elements(bez_bordelu, return_dtype=str))
df = df.explode("020_q").with_columns(pl.col("020_q").map_elements(zjisti_vazbu, return_dtype=str).alias('vazba'))
df = df.explode('245_p').with_columns(pl.col('245_p').map_elements(bez_bordelu, return_dtype=str))
print(len(df))
df = df.filter(pl.col("rok") >= 1800)

df = df.filter((~pl.col("245_h").str.contains("grafika")) | pl.col("245_h").is_null()).unique(subset=["008","100_a","245_a","245_p"], keep="first")
print(len(df))

794838
705250


In [403]:
aut = pl.read_parquet(os.path.join("data","aut_vyber.parquet"))
cesi = aut.explode("370_c").filter(pl.col("370_c").str.contains("Česk")).explode("100_7").select(pl.col("100_7")).to_series().to_list()
len(cesi)

363993

## Budeme brát do úvahy pouze autorstvo vícenásobné!

In [193]:
vicenasobni = df.group_by('100_7').len().filter(pl.col('len') >= 3).select(pl.col('100_7')).to_series().to_list()
len(vicenasobni)

53490

In [195]:
df = df.filter(pl.col('100_7').is_in(vicenasobni))

In [197]:
len(df)

532717

## Kolik % autorstva ten rok skončilo

In [7]:
df.sample(20)

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,leader,008,020_q,020_c,020_a,020_z,022_a,022_y,022_z,022_ind1,022_l,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,300_a,300_b,300_c,300_e,300_f,300_3,655_ind2,655_a,655_7,655_2,655_ind1,655_x,655_z,655_y,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_m,700_n,700_k,700_r,700_p,700_o,700_s,700_j,700_6,700_x,700_e,700_f,700_5,700_g,rok,stran,vazba
str,str,str,list[str],str,str,list[str],str,str,str,str,str,str,list[str],list[str],list[str],str,list[str],list[str],str,str,str,str,str,str,str,list[str],str,str,str,str,list[str],list[str],list[str],list[str],str,str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],i64,i64,str
"""1""","""Navarová, Zuzana,""","""jn20000710455""","[""aut""]","""1959-2004""",null,null,null,null,"""nkc20172954505""",""" nam a22 i 4500""","""171110s2017 xr ac e 0…","""(vázáno) :""","[""Kč 289,00""]","[""978-80-7577-333-3""]",null,null,null,null,null,null,"""1""","""0""","""Andělská počta""",null,"""Zuzana Navarová""",null,null,null,null,null,"[""167 stran :""]","[""ilustrace, portréty ;""]","[""20 cm""]",null,null,null,"[""7"", ""7"", … ""9""]","[""písňové texty"", ""vzpomínky"", … ""interviews""]","[""fd133190"", ""fd133830"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2017,167,"""pevná"""
"""1""","""Marhoul, Alois,""","""jn99240000678""","[""aut""]","""1951-2024""",null,null,null,null,"""nkc20071708144""",""" nam a22 a 4500""","""070219s2006 xr a g 0…","""(váz.)""",null,"[""80-86078-64-7""]",null,null,null,null,null,null,"""1""","""0""","""Kam jablko nepadá""",null,"""Alois Marhoul ; ilustroval Mir…",null,null,null,null,null,"[""91 s. :""]","[""il. ;""]","[""15 x 21 cm""]",null,null,null,"[""7"", ""9""]","[""česká poezie"", ""Czech poetry""]","[""fd133958"", null]","[""czenas"", ""eczenas""]",null,null,null,null,"[""1""]","[""Huptych, Miroslav,""]","[""ill""]","[""1952-""]","[""jn19990009850""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2006,91,"""pevná"""
"""1""","""Casey, Jane""","""xx0164515""","[""aut""]","""1977-""","""(Jane E.),""",null,null,null,"""nkc20132518003""",""" cam a22 a 4500""","""131031s2013 xr g 0…","""(váz.)""",null,"[""978-80-257-0981-8""]",null,null,null,null,null,null,"""1""","""0""","""Pomsta""",null,"""Jane Caseyová ; přeložila Jark…",null,null,null,null,null,"[""429 s. ;""]",null,"[""24 cm""]",null,null,null,"[""7"", ""7"", … ""9""]","[""anglicky psané romány"", ""detektivní romány"", … ""detective novels""]","[""fd131805"", ""fd132010"", … null]","[""czenas"", ""czenas"", … ""eczenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2013,429,"""pevná"""
"""1""","""Velsovský, Anatol""",null,"[""com""]",null,null,null,null,null,"""bknaus15727""",""" nam a22 1 4500""","""020918s1955 xr d p |…",null,null,null,null,null,null,null,null,null,"""1""","""0""","""Válcování II""","""Učební text pro 3. roč. průmys…","""Sest. Anatol Velsovský""","[""1. část /""]",null,null,null,null,"[""110 s. :""]","[""il., tab. ;""]","[""4°""]",null,null,null,"[""7""]","[""učebnice středních škol""]","[""fd133771""]","[""czenas""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1955,110,null
"""1""","""Seifert, Jaroslav,""","""jk01110657""","[""aut""]","""1901-1986""",null,null,null,null,"""

In [75]:
df = df.join(df.group_by("100_a").agg(pl.col("rok").max().alias("posledni_kniha")), on="100_a", how="left")

In [79]:
df.filter(pl.col('rok') == pl.col('posledni_kniha'))

100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,001,leader,008,020_q,020_c,020_a,020_z,022_a,022_y,022_z,022_ind1,022_l,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,300_a,300_b,300_c,300_e,300_f,300_3,…,655_7,655_2,655_ind1,655_x,655_z,655_y,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_m,700_n,700_k,700_r,700_p,700_o,700_s,700_j,700_6,700_x,700_e,700_f,700_5,700_g,rok,stran,vazba,posledni_kniha,posledni_kniha_right
str,str,str,list[str],str,str,list[str],str,str,str,str,str,str,list[str],list[str],list[str],str,list[str],list[str],str,str,str,str,str,str,str,list[str],str,str,str,str,list[str],list[str],list[str],list[str],str,str,…,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],i64,i64,str,i64,i64
"""1""","""Cenková, Pavla,""","""jx20101222003""","[""aut""]","""1979-""",null,null,null,null,"""nkc20243584877""",""" nam a22 i 4500""","""240123s2023 xr aeh e f 0…","""(brožováno)""",null,"[""978-80-87967-36-2""]",null,null,null,null,null,null,"""1""","""0""","""Portrét ulice Pellicova""",null,"""autoři textů Pavla Cenková, Al…",null,null,null,null,null,"[""26 nečíslovaných stran :""]","[""ilustrace (převážně barevné), plány, faksimile ;""]","[""18 x 25 cm""]",null,null,null,…,"[""fd133154"", null]","[""czenas"", ""eczenas""]",null,null,null,null,"[""1"", ""1"", ""1""]","[""Homola, Aleš,"", ""Kruntorád, Matěj,"", ""Procházková, Marta,""]","[""aut"", ""aut"", ""aut""]","[""1975-"", ""1986-"", ""1966-""]","[""mzk2006342918"", ""mzk2013768871"", ""mzk2009517462""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2023,26,"""brožovaná""",2023,2023
"""1""","""Šebestík, Ladislav,""","""jk01122626""","[""aut""]","""-2002""",null,null,null,null,"""ck8506789""",""" nam a22 4500""","""851007s1984 xr u0…","""(Brož.)""",null,null,null,null,null,null,null,null,"""1""","""0""","""Znak městečka Modřic""",null,"""Ladislav Šebestík""",null,null,null,null,null,"[""21 s. :""]","[""il. ;""]","[""21 cm""]",null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1984,21,"""brožovaná""",1984,1984
"""1""","""Gammond, Peter""","""jn19981001311""","[""aut""]",null,null,null,null,null,"""cpk19960167438""",""" nam a22 a 4500""","""970219s1996 xr e 0…","""(brož.)""",null,"[""80-7197-011-5""]",null,null,null,null,null,null,"""1""","""0""","""Jak blufovat o hudbě""",null,"""Peter Gammond ; [z angličtiny …",null,null,null,null,null,"[""62 s. ;""]",null,"[""18 cm""]",null,null,null,…,"[""fd133209""]","[""czenas""]",null,null,null,null,"[""1""]","[""Kvapil, Jan,""]","[""trl""]","[""1943-""]","[""jn20001031025""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1996,62,"""brožovaná""",1996,1996
"""1""","""Hansen Čechová, Barbara,""","""xx0033061""","[""aut""]","""1979-""",null,null,null,null,"""nkc20112255847""",""" nam a22 a 4500""","""111122s2011 xr a e f 0…","""(brož.)""",null,"[""978-80-7430-059-2""]",null,null,null,null,null,null,"""1""","""0""","""Nápadník pro výuku dle učebníc…",null,"""Barbara Čechová, Matěj Seifert…",null,null,null,null,null,"[""183 s. :""]","[""il. (převážně barev.) ;""]","[""30 cm""]",null,null,null,…,"[""fd132826"", null]","[""czenas"", ""eczenas""]",null,null,null,null,"[""1"", ""1""]","[""Seifert, Matěj"", ""Vedralová, Andrea""]","[""aut"", ""aut""]",null,"[""jo2011672137"", ""jo2011672139""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2011,183,"""brožovaná""",2011,2011
"""1""","""Mittermayerová, Marie""","""mzk20095

In [405]:
podil_koncicich = df.filter(pl.col('100_7').is_in(cesi)).filter(pl.col('rok') == pl.col('posledni_kniha')).group_by('rok').agg(pl.col('100_7').n_unique()).rename({'100_7':'skoncilo'}).join(df.filter(pl.col('100_7').is_in(cesi)).group_by('rok').agg(pl.col('100_7').n_unique()).rename({'100_7':'celkem'}), on='rok', how='left').with_columns((pl.col('skoncilo') / pl.col('celkem')).alias('podil')).sort(by='rok')
podil_koncicich

rok,skoncilo,celkem,podil
i64,u32,u32,f64
1805,1,6,0.166667
1813,1,6,0.166667
1816,1,12,0.083333
1818,1,14,0.071429
1822,3,22,0.136364
1823,1,22,0.045455
1824,1,20,0.05
1825,2,28,0.071429
1827,1,23,0.043478


In [407]:
podil_koncicich.filter(pl.col('rok').is_between(1900,2000)).sort(by='podil')

rok,skoncilo,celkem,podil
i64,u32,u32,f64
1905,25,708,0.035311
1954,56,1509,0.037111
1918,26,692,0.037572
1900,20,521,0.038388
1902,29,755,0.038411
1903,29,723,0.040111
1911,37,890,0.041573
1919,45,1049,0.042898
1901,28,648,0.04321


In [409]:
podil_koncicich.write_parquet(os.path.join('data','podily_konciciho_autorstva_cs.parquet'))

## Nejproduktivnější autorstvo nevydávané po historických zlomech

In [384]:
def nevysli_po(rok, obdobi):
    vysli_pred = df.filter(pl.col("rok").is_between(rok-obdobi,rok)).select(pl.col("100_7")).to_series().to_list()
    vysli_po = set(df.filter(pl.col("rok") > rok).select(pl.col("100_7")).to_series().to_list())
    ukonceni = [v for v in vysli_pred if v not in vysli_po]
    print(f"Filtru vyhovuje {len(ukonceni)} lidí.")
    return df.filter(pl.col("100_7").is_in(ukonceni)).group_by(["100_a","100_7"]).len().sort(by="len",descending=True).head(100)

In [386]:
nevysli_po(1918, 18)

Filtru vyhovuje 3677 lidí.


100_a,100_7,len
str,str,u32
"""Adámek, Karel,""","""jk01010079""",147
"""Dumek, Josef,""","""jk01023310""",88
"""Močnik, Franc,""","""jn20001103225""",73
"""Frič, Antonín,""","""jk01031862""",54
"""Gindely, Antonín,""","""jk01032434""",52
"""Zap, Karel Vladislav,""","""jk01152094""",40
"""Sobek, František,""","""jk01120224""",38
"""Štulc, Václav Svatopluk,""","""jk01131557""",38
"""Hanka, Václav,""","""jk01033420""",37


In [388]:
nevysli_po(1945, 5)

Filtru vyhovuje 2213 lidí.


100_a,100_7,len
str,str,u32
"""Batěk, Alexandr,""","""jk01011106""",322
"""Kapras, Jan,""","""jk01052930""",120
"""Andrlík, František Josef,""","""jk01010291""",108
"""Riedl Německobrodský, Ladislav…","""jk01102414""",100
"""Klecanda, Jan,""","""jk01060320""",97
"""Fořt, Karel,""","""jk01031626""",92
"""Procházka, František Serafínsk…","""jk01101127""",76
"""Vrána, Václav,""","""jk01150904""",73
"""Tisovský, T. E.""","""jk01132505""",71


In [390]:
nevysli_po(1989, 25)

Filtru vyhovuje 36384 lidí.


100_a,100_7,len
str,str,u32
"""Nejedlý, Zdeněk,""","""jk01082971""",251
"""Balák, Karel,""","""jk01010610""",211
"""Baudyš, Eduard,""","""jk01011141""",183
"""Balda, Jiří,""","""jk01010666""",165
"""Gottwald, Klement,""","""jk01032566""",157
"""Zápotocký, Antonín,""","""jk01152141""",131
"""Sokol Tůma, František,""","""jk01120356""",124
"""Šmilovský, Alois Vojtěch,""","""jk01130612""",122
"""Hostáň, Jan,""","""jk01042385""",114


In [392]:
nevysli_po(2000, 9)

Filtru vyhovuje 11905 lidí.


100_a,100_7,len
str,str,u32
"""Lenin, Vladimir Il‘jič,""","""jn19981001737""",339
"""Strong, Pitt,""","""jo2005267810""",323
"""Mark, William,""","""jo20241218643""",214
"""Vymazal, František,""","""jk01151194""",198
"""Schweigstill, Bohumil,""","""jk01111244""",191
"""Hais Týnecký, Josef,""","""jk01033015""",163
"""Marešová, Eva,""","""jo2008475985""",162
"""Vachek, Emil,""","""jk01141130""",150
"""Barner, Gerhard F.""","""jx20040908007""",122


## Normalizace

Nejdříve porovnáme všechna x-letá období ve 20. století.

In [215]:
%%time
odmlceni = []
for r in range(1967,1970):
    print(r)
    vychazeli_pred = df.filter(pl.col("rok") < r).select(pl.col("100_7")).to_series().to_list()
    vychazeli_po = set(df.filter(pl.col("rok") > (r + 18)).select(pl.col("100_7")).to_series().to_list())
    vychazeli_mezi = df.filter(pl.col("rok").is_between(r, r+18)).select(pl.col("100_7")).to_series().to_list()
    cancellovani = [v for v in vychazeli_pred if (v in vychazeli_po) and (v not in vychazeli_mezi)]
    odmlceni.append({'rok': r, 'podil': len(cancellovani) / len(vychazeli_pred)})

1967
1968
1969
CPU times: total: 7min 37s
Wall time: 7min 50s


In [216]:
pl.DataFrame(odmlceni)

rok,podil
i64,f64
1967,0.11093
1968,0.111395
1969,0.115511


In [314]:
delky_pauzy = [5,10,15,20]
odmlceni = []
for r in range(1820,2020):
    rocni = {'rok': r}
    for delka_pauzy in delky_pauzy:
        print(f"{r} + {delka_pauzy} let")
        vychazeli_pred = set(df.filter(pl.col("rok") < r).select(pl.col("100_7")).to_series().to_list())
        vychazeli_po = set(df.filter(pl.col("rok") > (r + delka_pauzy)).select(pl.col("100_7")).to_series().to_list())
        vychazeli_mezi = set(df.filter(pl.col("rok").is_between(r, r+delka_pauzy)).select(pl.col("100_7")).to_series().to_list())
        cancellovani = vychazeli_pred.intersection(vychazeli_po).difference(vychazeli_mezi)
        rocni[f'podil_vsech_{delka_pauzy}'] =  len(cancellovani) / len(vychazeli_pred)
        vychazeli_pred = set(df.filter(pl.col("rok") < r).filter(pl.col('100_7').is_in(cesi)).select(pl.col("100_7")).to_series().to_list())
        vychazeli_po = set(df.filter(pl.col("rok") > (r + delka_pauzy)).filter(pl.col('100_7').is_in(cesi)).select(pl.col("100_7")).to_series().to_list())
        vychazeli_mezi = set(df.filter(pl.col("rok").is_between(r, r+delka_pauzy)).filter(pl.col('100_7').is_in(cesi)).select(pl.col("100_7")).to_series().to_list())
        cancellovani = vychazeli_pred.intersection(vychazeli_po).difference(vychazeli_mezi)
        rocni[f'podil_cechu_{delka_pauzy}'] =  len(cancellovani) / len(vychazeli_pred)
    odmlceni.append(rocni)

1820 + 5 let
1820 + 10 let
1820 + 15 let
1820 + 20 let
1821 + 5 let
1821 + 10 let
1821 + 15 let
1821 + 20 let
1822 + 5 let
1822 + 10 let
1822 + 15 let
1822 + 20 let
1823 + 5 let
1823 + 10 let
1823 + 15 let
1823 + 20 let
1824 + 5 let
1824 + 10 let
1824 + 15 let
1824 + 20 let
1825 + 5 let
1825 + 10 let
1825 + 15 let
1825 + 20 let
1826 + 5 let
1826 + 10 let
1826 + 15 let
1826 + 20 let
1827 + 5 let
1827 + 10 let
1827 + 15 let
1827 + 20 let
1828 + 5 let
1828 + 10 let
1828 + 15 let
1828 + 20 let
1829 + 5 let
1829 + 10 let
1829 + 15 let
1829 + 20 let
1830 + 5 let
1830 + 10 let
1830 + 15 let
1830 + 20 let
1831 + 5 let
1831 + 10 let
1831 + 15 let
1831 + 20 let
1832 + 5 let
1832 + 10 let
1832 + 15 let
1832 + 20 let
1833 + 5 let
1833 + 10 let
1833 + 15 let
1833 + 20 let
1834 + 5 let
1834 + 10 let
1834 + 15 let
1834 + 20 let
1835 + 5 let
1835 + 10 let
1835 + 15 let
1835 + 20 let
1836 + 5 let
1836 + 10 let
1836 + 15 let
1836 + 20 let
1837 + 5 let
1837 + 10 let
1837 + 15 let
1837 + 20 let
1838 + 5 l

In [310]:
odmlceni

[{'rok': 1900,
  'podil_vsech_5': 0.27809081869406693,
  'podil_cechu_5': 0.23147340889276374,
  'podil_vsech_10': 0.17434537620152468,
  'podil_cechu_10': 0.14210985178727115,
  'podil_vsech_15': 0.12429565793834936,
  'podil_cechu_15': 0.1011333914559721,
  'podil_vsech_20': 0.09910507126284389,
  'podil_cechu_20': 0.08326068003487358},
 {'rok': 1901,
  'podil_vsech_5': 0.27562062380649266,
  'podil_cechu_5': 0.2270066889632107,
  'podil_vsech_10': 0.1769573520050923,
  'podil_cechu_10': 0.14423076923076922,
  'podil_vsech_15': 0.13112667091024824,
  'podil_cechu_15': 0.10702341137123746,
  'podil_vsech_20': 0.09961807765754296,
  'podil_cechu_20': 0.08486622073578595}]

In [315]:
pl.DataFrame(odmlceni)

rok,podil_vsech_5,podil_cechu_5,podil_vsech_10,podil_cechu_10,podil_vsech_15,podil_cechu_15,podil_vsech_20,podil_cechu_20
i64,f64,f64,f64,f64,f64,f64,f64,f64
1820,0.477778,0.363636,0.422222,0.327273,0.377778,0.290909,0.288889,0.2
1821,0.494845,0.409836,0.43299,0.344262,0.350515,0.262295,0.28866,0.213115
1822,0.523364,0.457143,0.420561,0.357143,0.327103,0.257143,0.271028,0.214286
1823,0.5,0.425,0.398305,0.325,0.305085,0.2375,0.20339,0.15
1824,0.473684,0.402174,0.368421,0.293478,0.308271,0.23913,0.210526,0.163043
1825,0.492857,0.419355,0.407143,0.322581,0.314286,0.236559,0.214286,0.16129
1826,0.543624,0.459184,0.436242,0.336735,0.342282,0.265306,0.261745,0.193878
1827,0.529412,0.435644,0.411765,0.306931,0.326797,0.247525,0.267974,0.188119
1828,0.56875,0.466667,0.45,0.352381,0.2875,0.2,0.275,0.180952


In [319]:
pl.DataFrame(odmlceni).with_columns(
                   pl.col("rok").map_elements(
                       lambda x: datetime.date(year=int(x), month=1, day=1), 
                       return_dtype=pl.Date
                   ).cast(pl.Datetime)).write_parquet(os.path.join('data',f'podil_autorstva_s_pauzou.parquet'))

In [321]:
odmlceni = pl.DataFrame(odmlceni).with_columns(
                   pl.col("rok").map_elements(
                       lambda x: datetime.date(year=int(x), month=1, day=1), 
                       return_dtype=pl.Date
                   ).cast(pl.Datetime))

In [269]:
import altair as alt

In [335]:
odmlceni.filter(pl.col("rok") < datetime.date(year = 2010, month=1, day=1))

rok,podil_vsech_5,podil_cechu_5,podil_vsech_10,podil_cechu_10,podil_vsech_15,podil_cechu_15,podil_vsech_20,podil_cechu_20
datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64
1820-01-01 00:00:00,0.477778,0.363636,0.422222,0.327273,0.377778,0.290909,0.288889,0.2
1821-01-01 00:00:00,0.494845,0.409836,0.43299,0.344262,0.350515,0.262295,0.28866,0.213115
1822-01-01 00:00:00,0.523364,0.457143,0.420561,0.357143,0.327103,0.257143,0.271028,0.214286
1823-01-01 00:00:00,0.5,0.425,0.398305,0.325,0.305085,0.2375,0.20339,0.15
1824-01-01 00:00:00,0.473684,0.402174,0.368421,0.293478,0.308271,0.23913,0.210526,0.163043
1825-01-01 00:00:00,0.492857,0.419355,0.407143,0.322581,0.314286,0.236559,0.214286,0.16129
1826-01-01 00:00:00,0.543624,0.459184,0.436242,0.336735,0.342282,0.265306,0.261745,0.193878
1827-01-01 00:00:00,0.529412,0.435644,0.411765,0.306931,0.326797,0.247525,0.267974,0.188119
1828-01-01 00:00:00,0.56875,0.466667,0.45,0.352381,0.2875,0.2,0.275,0.180952


In [343]:
alt.Chart(odmlceni.filter(pl.col("rok") < datetime.date(year = 2010, month=1, day=1)).to_pandas()).mark_line().encode(alt.Y("podil_cechu_10:Q"),alt.X('rok:T'))

alt.Chart(...)

In [347]:
alt.Chart(odmlceni.filter(pl.col("rok").is_between(datetime.date(year = 1900, month=1, day=1), datetime.date(year = 2000, month=1, day=1))).to_pandas()).mark_line().encode(alt.Y("podil_cechu_15:Q"),alt.X('rok:T'))

alt.Chart(...)

In [277]:
alt.Chart(pl.DataFrame(odmlceni).to_pandas()).mark_line().encode(alt.Y("podil_vsech:Q"),alt.X('rok:T'))

alt.Chart(...)

## Znormalizovaní

In [280]:
pred_normalizaci = set(df.filter(pl.col("rok") <= 1972).select(pl.col("100_7")).to_series().to_list())
po_normalizaci = set(df.filter(pl.col("rok") >= 1988).select(pl.col("100_7")).to_series().to_list())
behem_normalizace = set(df.filter(pl.col("rok").is_between(1973,1987)).select(pl.col("100_7")).to_series().to_list())

In [395]:
znormalizovani = pred_normalizaci.intersection(po_normalizaci).difference(behem_normalizace)

In [401]:
df.drop_nulls(subset=['700_7','700_a','700_4']).explode(['700_7','700_a','700_4']).filter(pl.col(["700_7"]).is_in(znormalizovani)).filter(pl.col("rok").is_between(1973,1987)).group_by(['700_7','700_a','700_4']).len().sort(by='len',descending=True)

700_7,700_a,700_4,len
str,str,str,u32
"""jk01083128""","""Němeček, Jaroslav,""","""ill""",95
"""jk01031767""","""Franta, Karel,""","""ill""",37
"""jk01101873""","""Rada, Miroslav,""","""ill""",26
"""jk01093059""","""Pilař, Radek,""","""ill""",25
"""jk01152086""","""Zaorálek, Jaroslav,""","""trl""",16
"""jk01111589""","""Skoumal, Aloys,""","""trl""",13
"""mzk2004148400""","""Svoboda, Pavel,""","""aut""",13
"""jk01052538""","""Kahounová, Jana,""","""aut""",13
"""jk01021228""","""Čemus, Otakar,""","""ill""",13


In [378]:
df.filter(pl.col("100_7").is_in(znormalizovani) & pl.col("100_7").is_in(cesi)).group_by(["100_a",'100_7']).len().sort(by='len',descending=True)

100_a,100_7,len
str,str,u32
"""Javořická, Vlasta,""","""jk01051344""",462
"""Masaryk, Tomáš Garrigue,""","""jk01080472""",363
"""Pittnerová, Vlasta,""","""jk01093291""",338
"""Svoboda, F. X.,""","""jk01121960""",229
"""Šťastný, Alfons Bohumil,""","""jk01131099""",225
"""Zahradník-Brodský, Bohumil,""","""jk01151969""",223
"""Vymazal, František,""","""jk01151194""",198
"""Schweigstill, Bohumil,""","""jk01111244""",191
"""Krásnohorská, Eliška,""","""jk01062911""",185


In [380]:
df.filter(pl.col("100_7").is_in(znormalizovani)).group_by(["100_a",'100_7']).len().sort(by='len',descending=True)

100_a,100_7,len
str,str,u32
"""Courths-Mahler, Hedwig,""","""jn19990001513""",648
"""Javořická, Vlasta,""","""jk01051344""",462
"""Masaryk, Tomáš Garrigue,""","""jk01080472""",363
"""Clifton, Léon""","""xx0111518""",353
"""Pittnerová, Vlasta,""","""jk01093291""",338
"""Strong, Pitt,""","""jo2005267810""",323
"""Steiner, Rudolf,""","""jn19990009728""",243
"""Svoboda, F. X.,""","""jk01121960""",229
"""Šťastný, Alfons Bohumil,""","""jk01131099""",225


## Zúnorovaní

In [370]:
pred_unorem = set(df.filter(pl.col("rok") <= 1948).select(pl.col("100_7")).to_series().to_list())
po_revoluci = set(df.filter(pl.col("rok") >= 1988).select(pl.col("100_7")).to_series().to_list())
v_sedesatkach = set(df.filter(pl.col("rok").is_between(1960,1969)).select(pl.col("100_7")).to_series().to_list())
behem_socialismu = set(df.filter(pl.col("rok").is_between(1949,1987)).select(pl.col("100_7")).to_series().to_list())
behem_padesatek = set(df.filter(pl.col("rok").is_between(1949,1959)).select(pl.col("100_7")).to_series().to_list())

In [366]:
zunorovani = pred_unorem.intersection(po_revoluci).difference(behem_socialismu)
df.filter(pl.col("100_7").is_in(zunorovani)).group_by(["100_a",'100_7']).len().sort(by='len',descending=True)

100_a,100_7,len
str,str,u32
"""Courths-Mahler, Hedwig,""","""jn19990001513""",648
"""Javořická, Vlasta,""","""jk01051344""",462
"""Clifton, Léon""","""xx0111518""",353
"""Strong, Pitt,""","""jo2005267810""",323
"""Steiner, Rudolf,""","""jn19990009728""",243
"""Šťastný, Alfons Bohumil,""","""jk01131099""",225
"""Zahradník-Brodský, Bohumil,""","""jk01151969""",223
"""Vymazal, František,""","""jk01151194""",198
"""Dostál, Alois,""","""jk01022773""",179


In [382]:
df.filter(pl.col("100_7").is_in(zunorovani) & pl.col('100_7').is_in(cesi)).group_by(["100_a",'100_7']).len().sort(by='len',descending=True)

100_a,100_7,len
str,str,u32
"""Javořická, Vlasta,""","""jk01051344""",462
"""Šťastný, Alfons Bohumil,""","""jk01131099""",225
"""Zahradník-Brodský, Bohumil,""","""jk01151969""",223
"""Vymazal, František,""","""jk01151194""",198
"""Dostál, Alois,""","""jk01022773""",179
"""Marešová, Eva,""","""jo2008475985""",162
"""Caha, Arnošt,""","""jk01020519""",159
"""Medek, Rudolf,""","""jk01081030""",155
"""Hrnčíř, František,""","""jk01042782""",137


In [372]:
zunorovani2 = pred_unorem.intersection(v_sedesatkach).difference(behem_padesatek)
df.filter(pl.col("100_7").is_in(zunorovani2)).group_by(["100_a",'100_7']).len().sort(by='len',descending=True)

100_a,100_7,len
str,str,u32
"""Masaryk, Tomáš Garrigue,""","""jk01080472""",363
"""Grey, Zane,""","""jn19990002874""",237
"""Gardner, Erle Stanley,""","""jn19981001320""",234
"""Foglar, Jaroslav,""","""jk01031485""",220
"""Schweigstill, Bohumil,""","""jk01111244""",191
"""Beneš, Edvard,""","""jk01011576""",161
"""Johns, William Earl,""","""jn19990004054""",128
"""Chmelař, František,""","""jk01050240""",107
"""Karásek ze Lvovic, Jiří,""","""jk01052982""",100
